In [1]:
# 'use_log'와 'customer_join' 테이블의 대략적인 형태와 결측치 파악
uselog = pd.read_csv("./pyda100-master/Chapter_4/use_log.csv")
display(uselog.head())
display(uselog.isna().sum())

customer = pd.read_csv("customer_join.csv")
display(customer.head())
display(customer.isna().sum())


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


log_id         0
customer_id    0
usedate        0
dtype: int64

,customer_id,gender,start_date,end_date,is_deleted,class,class_name,campaign_id,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,F,2015-05-01,NaN,0,C01,0_종일,CA1,2_일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,M,2015-05-01,NaN,0,C01,0_종일,CA1,2_일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,M,2015-05-01,NaN,0,C01,0_종일,CA1,2_일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,F,2015-05-01,NaN,0,C01,0_종일,CA1,2_일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,F,2015-05-01,NaN,0,C03,2_야간,CA1,2_일반,3.916667,4.0,6,1,1,2019-04-30,47


customer_id             0
gender                  0
start_date              0
end_date             2842
is_deleted              0
class                   0
class_name              0
campaign_id             0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

In [2]:
# 'mean', 'median', 'max', 'min', 'membership_period'변수를 클러스터링을 위한 독립변수로 선정
customer_variables_for_clustering = customer.loc[:, ["mean", "median", "max", "min", "membership_period"]]
customer_variables_for_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


In [3]:
'''
'K-mean'를 활용한 클러스터링 실시
'''

# 사용될 독립변수들에 표준화 실시
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customer_scaled = sc.fit_transform(customer_variables_for_clustering)
print(customer_scaled, "\n")

# 클러스터링 실시
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(customer_scaled)

# 클러스터링 결과 확인
print("clustering 결과 :", clusters.labels_)
print("clustering 결과 내 요소 개수 :", len(clusters.labels_))
print("clustering 결과 내 고유 요소 개수 :", len(set(list(clusters.labels_))))
print("훈련 데이터 셋 내 데이터 개수 :", len(customer_scaled), "\n")

# 클러스터링 결과를 'customer_variables_for_clustering'데이터 셋에 컬럼으로 추가
customer_variables_for_clustering["cluster"] = clusters.labels_
customer_variables_for_clustering.head()

[[-0.28120614 -0.13367635  0.08117747 -0.5336197   2.03402524]
 [-0.14054502 -0.13367635 -0.37992814 -0.02114919  2.03402524]
 [-0.42186727 -0.13367635 -0.84103375 -0.02114919  2.03402524]
 ...
 [ 1.50050144  1.46662237  0.08117747  2.54120338 -1.28811491]
 [ 3.18843493  3.06692109  1.46449428  4.07861492 -1.28811491]
 [ 2.62579043  2.53348819  1.00338868  3.56614441 -1.28811491]] 

clustering 결과 : [1 1 1 ... 0 0 0]
clustering 결과 내 요소 개수 : 4192
clustering 결과 내 고유 요소 개수 : 4
훈련 데이터 셋 내 데이터 개수 : 4192 



C:\Users\my\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\my\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,mean,median,max,min,membership_period,cluster
0,4.833333,5.0,8,2,47,1
1,5.083333,5.0,7,3,47,1
2,4.583333,5.0,6,3,47,1
3,4.833333,4.5,7,2,47,1
4,3.916667,4.0,6,1,47,1


In [4]:
'''
클러스터링 결과 분석
'''
# 클러스터별 고객수 확인
customer_variables_for_clustering.columns = ["월 방문횟수 평균값", "월 방문횟수 중앙값", "월 방문횟수 최댓값", "월 방문횟수 최솟값", "회원기간", "cluster"]
customer_variables_for_clustering.head()
customer_variables_for_clustering.groupby("cluster").count()

,mean,median,max,min,membership_period
cluster,,,,,
0,841,841,841,841,841
1,1248,1248,1248,1248,1248
2,771,771,771,771,771
3,1332,1332,1332,1332,1332


In [5]:
# 클러스터별 평균값 확인
customer_variables_for_clustering.groupby("cluster").mean()

,mean,median,max,min,membership_period
cluster,,,,,
0,8.061076,8.046373,10.015458,6.171225,7.027348
1,4.677436,4.671074,7.233173,2.153045,36.925481
2,3.065504,2.900130,4.783398,1.649805,9.276265
3,5.537658,5.388889,8.753754,2.702703,14.876126


In [11]:
'''
'주성분 분석'을 통한 차원 축소를 이용하여, 클러스터링 결과 시각화 실시
'''

# 'customer_variables_for_clustering'에 PCA 실시
from sklearn.decomposition import PCA
X = customer_variables_for_clustering
pca = PCA(n_components=2)
pca.fit(X)
x_pca = pca.transform(X)

# pca 결과를 Dataframe으로 생성한 뒤, 해당 Dataframe에 'cluster'컬럼 추가
pca_df = pd.DataFrame(x_pca)
pca_df["cluster"] = customer_variables_for_clustering["cluster"]
pca_df.head()

,0,1,cluster
0,28.166818,1.223570,1
1,28.134671,1.268319,1
2,28.178408,0.499804,1
3,28.207873,0.447258,1
4,28.330494,-1.224347,1
